In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install lion-pytorch

In [4]:
%pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 8.6 MB/s eta 0:00:00


In [5]:
import torch

from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from lion_pytorch import Lion
from utility.step_lr import StepLR
from optimizers_impl.gsam import CosineScheduler

In [6]:
def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions.logits, targets, smoothing=0.0001)
        loss.mean().backward()

        optimizer.step()
        optimizer.zero_grad()

        with torch.no_grad():
            correct = torch.argmax(predictions.logits, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            # scheduler(epoch)
            scheduler.step()


In [7]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.0001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [8]:
if __name__ == "__main__":

    num_epochs = 50
    # model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'

    # model_name = model_name_path + 'inceptionv3-lion'
    model_name =  'inceptionv3-lion'

    initialize(seed=111)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True).to(device)
    model.num_classes = 2
    model.eval()

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=299)

    optimizer = Lion(model.parameters(), lr=0.1, weight_decay=0.003)

    # optimizer = Lion(model.parameters(), lr=0.2, weight_decay=0.001) trenuj takie jak jest
#     optimizer = Lion(model.parameters(), lr=0.004, weight_decay=0.05)

    scheduler = CosineScheduler(T_max=num_epochs, max_value=0.0005, min_value=0.0, optimizer=optimizer)
    # scheduler = StepLR(optimizer, 0.0005, 50)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:02<00:00, 50.5MB/s] 


100
300
880
┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      4.7487  │      0.42 %  ┃   4.762e-04  │       00:25  ┃6165897.1733  │     50.00 %  ┃
┃           1  ┃      2.2377  │      0.50 %  ┃   3.840e-04  │       00:04  ┃    483.9856  │     50.00 %  ┃
┃           2  ┃      1.8576  │      0.50 %  ┃   2.500e-04  │       00:04  ┃      0.9303  │     50.00 %  ┃
┃           3  ┃      1.7256  │      0.50 %  ┃   1.160e-04  │       00:04  ┃      0.8121  │     50.00 %  ┃
┃           4  ┃      1.4863  │      0.50 %  ┃   2.379e-05  │       00:04  ┃      0.8074  │     50.00 %  ┃
┃           5  ┃      1.4509  │      0.50 %  ┃   1.971e-06  │       00:04  ┃      0.8072  │     50.00 %  ┃
┃           6  ┃      1.6650  │      0.50 %  ┃   5.737e-05  │       00:04  ┃      0.8129  │     50.00 %  ┃
┃           7  ┃      1.4489  │      0.50 %  ┃   1.727e-04  │       00:04  ┃      0.8045  │     50.00 %  ┃
┃           8  ┃      1.2366  │      0.51 %  ┃   3.122e-04  │       00:04  ┃      0.8371  │     50.00 %  ┃
┃           9  ┃      1.1414  │      

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


tensor(0.9346, device='cuda:0')
tensor(0.9300, device='cuda:0')
tensor(0.8772, device='cuda:0')
tensor(0.8600, device='cuda:0')
tensor(1., device='cuda:0')
